<a href="https://colab.research.google.com/github/cobridi/detecao-anomalias-emissoes/blob/main/dete%C3%A7%C3%A3o_anomalia_emissoes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introdução

Este Notebook faz parte do trabalho final do curso BI MASTER Turma 2019.03 do aluno Claudio Bridi, orientado pela Prof. Evelyn.

# Objetivo: 

Criar um algoritmo para detectar anomalias em série temporal de emissões mensais de CO2-equivalente das fontes de uma UEP (Unidade Estacionária de Produção de Óleo e Gás), no período de 2015 a 2019.

# Material
- Google Colaboratory
- Planilha de dados brutos contendo "nome da instalação", "tipo de fonte", "código da fonte", "Ano", "Mês", "CO2 emitido", em formatos csv e excel
- Git Hub
- Arquivo com código em python da disciplina CONF: deteçãodeanomaliasemseries.ipynb

# Aplicabilidade:
Este algoritmo poderá ser utilizado na industria de óleo e gás, mais especificamente, e também para empresas que precisem registrar, inventariar, divulgar e trabalhar com dados de emissões de gases de efeito estufa. Para detecção e correção das anomalias em dados, na maioria das vezes inseridos manualmente em sistema ou planilha específica, também costumamente são verificados no olho por algum profissional todos o anos, antes de comporem as informações do inventário e relatório de emissões, divulgado tanto internamente quanto para stakeholders externos. Isto é, por serem muitos dados e de muitas fontes, anomalias podem passar despercebidas, prejudicando a análise de dados, atendimento de compromissos assumidos e perda de credibilidade.

# Orientação:
Esta ideia surgiu na disciplina de Confiabilidade quando estudamos a detecção de anomalias em séries temporais de dados de precipitação em NY. Pensei em como ponto de partida utilizar o mesmo algorítimo e avançar a partir de então. #

# Prazo:
Finalizado no GitHub até 30/12/2020.


# Detecção de anomalias em Séries Temporais de Emissões de Gases de Efeito Estufa por equipamentos de uma plataforma de petróleo

### Planejamento por etapas:

*  Carregar, preparar e analisar dados
*  Pré-Processar os dados
*  Controle estatístico do processo (CEP) para detectar anomalias*
*  Modelos Autorregressivos
*  Avaliação dos modelos
*  Conclusão Final









## Importando



In [ ]:
%matplotlib inline
#import warnings
#warnings.filterwarnings('ignore', category=FutureWarning)
import sys
import datetime as dt
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import random
import seaborn as sns
from scipy import stats
import statsmodels as ss
import statsmodels.api as sm
from statsmodels.tsa.arima_model import ARIMA
from datetime import datetime
from dateutil.parser import parse

## Verificando as versões do Python e Bibliotieca

In [ ]:
packages = [matplotlib, np, pd]

msg = f"""
Python Version: {sys.version}

library .      version
-------        -------"""
print(msg)

for package in packages:
    print(f"{package.__name__:11}    {package.__version__:>7}")

# Seção 1: Carregar, Preparar e Analisar os dados

In [ ]:
df_emissions = pd.read_excel('Platco2e2015_2019.xlsx', header=7, index_col=0)
df_emissions.head()

Queremos duas coisas: Ter uma nova coluna que indique o ano e mês do registro de emissão (2019-10, por exemplo), para que a série de dados fique na sequência. Posteriormente vamos separar os dataframes por tipo de fonte ou Fonte Emissora, para que cada fonte seja uma série de dados.

In [ ]:
df_emissions['11 - Tipo de Fonte'].value_counts()

In [ ]:
df_emissions['11 - Tipo de Fonte'].describe()

In [ ]:
df_emissions['14 - Fonte Emissora'].value_counts()

In [ ]:
df_emissions['14 - Fonte Emissora'].describe()

Como visto na análise acima, a coluna '14 - Fonte Emissora' conta com 24 tipo de fontes. Já a coluna '11 - Tipo de Fonte' conta com 8 tipos. Minha ideia preliminar era realizar a análise de anomalia por Tipo de Fonte, no entanto vi que o mesmo tipo de fonte conta com mais de uma fonte emissora e os dados variam muito. Portanto a Análise deverá ser realizada po Fonte Emissora.

In [ ]:
df_emissions.info()

A coluna '20 - Mês/Ano' está como tipo 'object' e queremos que a mesma fique com o tipo de data e em ordem crescente para a correta análise da série de dados .

Vamos dividir a coluna "20 - Mês/Ano" e passar só o valor do mês para uma nova coluna:

In [ ]:
divisao = df_emissions['20 - Mês/Ano'].str.split(' ')

In [ ]:
divisao.head()

In [ ]:
mes = divisao.str.get(0)

In [ ]:
df_emissions['Mês'] = mes

In [ ]:
df_emissions.head()

Agora, vamos substituir os meses do formato com letras para números inteiros:

In [ ]:
df_emissions['Mês'].replace({'JAN':"01", "FEV":"02", 'MAR':'03', 'ABR':'04', 'MAI':'05', 'JUN':'06', 'JUL':'07', 'AGO':'08', 'SET':'09', 'OUT':'10', 'NOV':'11', 'DEZ':'12'}, inplace=True)

In [ ]:
df_emissions.head()

Vamos juntar em uma só coluna o ano e mês em formato "numérico"

In [ ]:
df_emissions['Ano-Mes'] = df_emissions['21 - Ano civil'].astype(str).add('-') + df_emissions['Mês']

In [ ]:
df_emissions.head()

In [ ]:
df_emissions.info()

Como a data está em formato de série, tenho que convertê-la para date:

In [ ]:
type(df_emissions['Ano-Mes'])

In [ ]:
df_emissions['Ano-Mes'] = pd.to_datetime(df_emissions['Ano-Mes'])

In [ ]:
df_emissions.head()

In [ ]:
df_emissions.info()

Colocando os dados em ordem crescente de data:

In [ ]:
df_emissions = df_emissions.sort_values('Ano-Mes')

In [ ]:
df_emissions.head()

Agora, vamos criar um novo dataframe com a primeira Fonte Emissora que vamos analisar: T-Z-1235001, que conta com 60 registros, isto é, registros mensais ao longo dos 5 anos da série histórica..

In [ ]:
df_fonte1= df_emissions.loc[df_emissions['14 - Fonte Emissora']=='T-Z-1235001']

In [ ]:
df_fonte1.head()

Vale a pena aqui observar que a Fonte Emissora T-Z-1235001 representa todos os 60 registros do Tipo de Fonte MEA/DEA, portanto estaremos avaliando todos os registros deste tipo de fonte.

Agora, vamos separar apenas as colunas com os datos que precisamos analisar: 'Ano-Mes' e os dados de emissões 'CO2e - AR4 (Mg)'

In [ ]:
df_fonte1_ = pd.DataFrame(df_fonte1, columns=['Ano-Mes','CO2e - AR4 (Mg)'])

In [ ]:
df_fonte1_.head()

Até tentei escoder o coluna de index com o código df_fonte1_.style.hide_index(), no entanto o mesmo parece modificar o dataframe e a leitura do mesmo não ficou igual. Não consegui utilizar df_fonte1_.head(), por exemplo. Aí abandonei esta alternativa e criei um outro arquivo .csv para a leitura dos dados ficarem como no exemplo de precipitações da aula, onde só havia duas colunas: 'tempo' 'registro de precipitação', sem index.

In [ ]:
#df_fonte1_teste = df_fonte1_.style.hide_index()

In [ ]:
#df_fonte1_teste.head()

Para ler os dados no modelo proposto precisamos do arquivo no formato .csv e apenas com as colunas de data e emissões. 

In [ ]:
df_fonte1_.to_csv('fonte1.csv', index=False)

# Leitura e primeira análise gráfica dos dados 

In [ ]:
newdf_fonte1 = pd.read_csv('fonte1.csv', parse_dates=True, index_col=0)

In [ ]:
newdf_fonte1.head()

In [ ]:
newdf_fonte1['CO2e - AR4 (Mg)'].describe()

In [ ]:
plt.figure(dpi=120)
newdf_fonte1.plot(ax=plt.gca())
plt.show();

Pela descrição dos dados bem como pelo gráfico plotado, é possível observar que os dados da fonte emissora 'T-Z-1235001' variam de 2279.143017 a 10384.143120, com média de 8008.698108 e desvio padrão de 2009.786665. Com os dados desta forma, não parece haver alguma anomalia bem como também parece ser muito difícil detectar caso haja.

# Análise de Anomalias 1.1: Gráfico de Controle

Pelas estatísticas resumidas e no gráfico provavelmente as anomalias estarão abaixo da média, que é de 8008.698108; portanto, precisamos apenas de um gráfico de controle unilateral. (Preciso descobrir um gráfico que veja acima e abaixo)

Como os dados de emissões não podem ser um número negativo, quase todos os valores possíveis acima da média estão no intervalo 8008.698108 <= emissão <= 10384.143120), Que estão quase todos dentro de um desvio padrão (2009.786665) Da média e, portanto, provavelmente não sejam anomalias (mas como dito acima, preciso de um gráfico que pegue acima e abaixo)

Portanto, vamos criar um gráfico de controle unilateral (para baixo que era o único exemplo que tinha)

In [ ]:
def control_plot(time_series, threshold):
    '''
    Creates a one-sided control plot from a time series
    (that is, plots threshold above the mean but not below)
    Also, returns list of points that exceed the threshold
    i.e., points for which the value > mean + threshold*(standard deviation)
    
    Args: 
        time_series: (pandas dataframe; index column is date in datetime format and  
        column 0 is data)
        threshold: z-score threshold for anomaly detection (float)

    Returns: 
        Control plot of time_series    
        anomalies: anomalies that exceed threshold (pandas dataframe)
    ''' 
    
    mean_= time_series.iloc[:,0].mean()
    stdev_= time_series.iloc[:,0].std()
    cutoff = mean_+threshold*stdev_
    plt.figure(dpi=140)
    time_series.plot(ax=plt.gca())
    plt.axhline(y=mean_, color='g', linestyle='--', label='mean')
    # Use threshold to plot line at threshold*stdev_ times away from the mean
    plt.axhline(y=cutoff, color='r', linestyle=':', label='threshold')
    plt.legend(loc='best')
    plt.title('Fonte1')
    plt.ylabel('Emissões CO2eq')
    
    # Create dataframe of anomalies that exceed the cutoff
    anomalies = time_series[time_series.values > cutoff]
    return anomalies

Defina um limite de 3 desvios padrão e plote os resultados.

In [ ]:
anomaly_fonte1 = control_plot(newdf_fonte1, 3)

Pelo gráfico gerado, não foi detectada nenhuma anomalia na Fonte 1 = T-Z-1235001. No entanto, como já indicado o mais provável é que a anomalia estivesse abaixo da média e não acima. Aqui que eu precisaria de uma ajudinha do orientador.

In [ ]:
print(anomaly_fonte1)

# Análise de Anomalias 1.2: Soma acumulada (CUSUM)

In [ ]:
def cusum(data, mean, shift, threshold):
    '''
   as somas cumulativas alta e baixa e use-as para detecção de anomalias.
     Uma anomalia é relatada se as somas cumulativas estiverem além de um determinado limite.
    
     Args:
         data: (uma série temporal como pandas dataframe; a coluna do índice é a data no formato datetime e
         coluna 0 são dados)
         mean: média dos dados ou outra média (flutuante)
         mudança: mudança normal nos dados; o desvio padrão é recomendado (flutuante)
         threshold: limite para classificar o ponto como anomalia (float)

     Devoluções:
         cusum: as somas cumulativas alta e baixa juntas Calcular com os dados (pandas dataframe)
         anomalias: anomalias acima e abaixo do limite (pandas dataframe)
    ''' 
    high_sum = 0.0
    low_sum = 0.0
    anomalies = [] 
    high_sum_final = []
    low_sum_final = []
    index_names = data.index
    data_values = data.values
    for index, item in enumerate(data_values):
        high_sum = max(0, high_sum + item - mean - shift)
        low_sum = min(0, low_sum + item - mean + shift)
        high_sum_final.append(high_sum)
        low_sum_final.append(low_sum)
        if high_sum > threshold or low_sum < -threshold:
            anomalies.append((index_names[index], item.tolist()))
    cusum = data
    cusum = cusum.assign(High_Cusum=high_sum_final, Low_Cusum=low_sum_final)
    return cusum, anomalies

In [ ]:
def cusum_plot(time_series, threshold):
    '''
    Plote as somas cumulativas alta e baixa e use-as para detecção de anomalias.
     Uma anomalia é relatada se as somas cumulativas estiverem além de um determinado limite.
    
     Args:
         time_series: (uma série temporal como pandas dataframe; a coluna de índice é date
         no formato datetime e a coluna 0 é data)
         threshold: limite para classificar o ponto como anomalia (float)

     Devoluções:
         Uma plotagem dos dados com a soma acumulada alta.
         cusum_results: as somas cumulativas alta e baixa, juntamente com dados
         e quaisquer anomalias acima e abaixo do limite (quadro de dados do pandas;
         da função cumsum)
    ''' 
    # Use a média e o desvio padrão de toda a série temporal
     # para calcular somas cumulativas
    mean_= time_series.iloc[:,0].mean()
    stdev_= time_series.iloc[:,0].std()
    
    # definir limiar em termos de desvio padrão
    cusum_results = cusum(time_series, mean_, stdev_, threshold*stdev_)
    ax=time_series.plot()
    ax.axhline(y=mean_, color='g', linestyle='--',label='average')
    ax.axhline(y=mean_+threshold*stdev_, color='r', linestyle='--',label='High threshold')
    ax.axhline(y=mean_-threshold*stdev_, color='r', linestyle='--',label='Low threshold')
    # Use threshold to plot line at threshold*stdev_ times away from the mean
    ax.scatter(x=cusum_results[0].index, y=cusum_results[0]['Low_Cusum'], 
             color='k', linestyle=':',label='Low Cusum')
    ax.scatter(x=cusum_results[0].index, y=cusum_results[0]['High_Cusum'], 
             color='y', linestyle=':',label='High Cusum')                 
    plt.legend(loc='lower right')
    plt.title('Fonte 1 Emissions - CUSUM')
    plt.ylabel('CO2 Emissions')
    plt.gcf().set_size_inches(8,6)
    plt.show()
    return cusum_results

Escolha do limiar: para o gráfico de controle, usamos 3 vezes o desvio padrão. Observe que o limite equivalente para CUSUM é * threshold * = 2, pois CUSUM inclui o * shift * (que escolhemos ser um desvio padrão) em seu cálculo da soma acumulada.

In [ ]:
anomaly_fonte1_cusum = cusum_plot(newdf_fonte1, 2);

Observe que LOW CUSUM parece detectar maiores anomalias. O resultado reflete a diferença nos algoritmos de CUSUM e Tabela de Controle. A tabela de controle está procurando pontos únicos e anômalos. CUSUM é sensível a alterações no comportamento dos dados. Como resultado, sinaliza pontos como anômalos até que o comportamento da série temporal retorne ao normal.

De fato, CUSUM é usado para detecção de pontos de mudança: encontrar quando a distribuição subjacente da série temporal mudou.

In [ ]:
print(anomaly_fonte1_cusum)

Não consegui imprimir apenas os dados que apresentaram apenas anomalias. No gráfico da aula o professo também só teve o dado visual. Outra dificuldade não solucionada foi na plotagem do CUSUM em cima do gráfico de emissões.

# Análise 1.3: modelos autorregressivos

Nos dois métodos anteriores, realizamos a análise no modo offline - tínhamos todos os dados de interesse em mãos. Como resultado, ao examinar um determinado ponto, poderíamos usar o passado e o futuro (com relação a esse ponto) para calcular estatísticas como a média e o desvio padrão.

Modelos autorregressivos são comumente usados para detecção de anomalias em fluxo. Para analisar séries temporais no modo de streaming - ou seja, à medida que os dados se tornam disponíveis.

Como linha de base para o modelo de regressão automática, podemos verificar o que um gráfico de controle revela como anomalias. Em contraste com o exemplo da precipitação, aqui podemos ter anomalias em ambos os lados da média, portanto modificamos * control_plot * para se tornar um gráfico de controle completo de dois lados.

In [ ]:
def control_plot_full(time_series, threshold):
    '''
    Cria um gráfico de controle frente e verso de uma série temporal
     (limiar de plotagem acima e abaixo da média)
     Além disso, retorna a lista de pontos que excedem o limite
     ou seja, pontos para os quais o valor> média + limite * (desvio padrão)
     e valor <média - limiar * (desvio padrão)
    
     Args:
         time_series: (pandas dataframe; a coluna de índice é a data no formato datetime e
         coluna 0 são dados)
         threshold: limite para detecção de anomalias (float)

     Devoluções:
         Gráfico de controle de time_series
         anomalias: anomalias que excedem o limite (pandas dataframe)
    ''' 
    
    mean_= time_series.iloc[:, 0].mean()
    stdev_= time_series.iloc[:, 0].std()
    time_series.plot()
    plt.axhline(y=mean_, color='g', linestyle='--',label='average')
    # Use threshold to plot line at threshold*stdev_ times away from the mean
    plt.axhline(y=mean_+threshold*stdev_, color='r', linestyle=':', label='high threshold')
    plt.axhline(y=mean_-threshold*stdev_, color='m', linestyle=':', label='low threshold')
    plt.legend(loc='upper right')
    plt.title('Fonte 1 Emissions')
    plt.ylabel('CO2eq Emissions')
    plt.gcf().set_size_inches(8,6)
    plt.show()
    
    # Create dataframe of anomalies that exceed the threshold
    anomaly_mask = (np.abs(time_series.values - mean_) > threshold*stdev_)
    anomalies = time_series[anomaly_mask]
    return anomalies

In [ ]:
control_plot_full(newdf_fonte1, 3)

Somente pelo resultado do gráfico não encontramos nenhuma anomalia.

In [ ]:
print(newdf_fonte1.index.inferred_freq)

O print acima serviu apenas para identificar a frequência dos registros: Frequência mensal

In [ ]:
newdf_fonte1.index.freq=newdf_fonte1.index.inferred_freq

In [ ]:
efonte1_sar=ARIMA(newdf_fonte1, order=(1,1,4)).fit()

efonte1_sar.summary()

Como no exemplo de aula, não discutiremos as estatísticas relatadas, exceto para dizer que ela inclui o desvio padrão dos resíduos (59.586	), que usaremos posteriormente.

Verificamos os diagnósticos para verificar se as premissas subjacentes ao modelo são atendidas e também para obter informações adicionais sobre a qualidade do ajuste. Isso é feito usando um gráfico Q-Q (verificando se os resíduos seguem uma distribuição normal), investigando os resíduos por padrões temporais e plotando um histograma dos resíduos

In [ ]:
# Get the predicted standard deviation. This is the 6.516 we saw earlier
sigma_pred = efonte1_sar.resid.std()
# Calculate the standardized residuals from the (regular) residuals
efonte1_std_resid = efonte1_sar.resid/sigma_pred

plt.title('Patterns in residual')
plt.plot(efonte1_std_resid);

In [ ]:
fig = plt.figure(figsize=(12,6))
ax = plt.subplot(121)
plt.title('Distribution of residuals')
sns.distplot(efonte1_std_resid.values, bins=50, ax=ax);
stats.probplot(efonte1_std_resid.values, dist='norm', sparams=(2.5,), plot=plt.subplot(122));

Para detecção de anomalias, focamos no gráfico superior: resíduos padronizados. O residual padronizado é o residual (a diferença entre o valor observado e o valor previsto) dividido pelo desvio padrão previsto (a raiz quadrada da variação prevista mencionada acima). É uma versão mais sofisticada do z-score.

Uma regra prática para detectar anomalias com resíduos padronizados: anomalias são pontos para os quais a magnitude dos resíduos padronizados é maior que 4. Vamos encontrar esses pontos.

In [ ]:
# Reportar as anomalias
anomaly_mask = np.abs(efonte1_std_resid) > 4
efonte1_anomalies = efonte1_std_resid[anomaly_mask]
print(efonte1_anomalies)

Nosso modelo encontrou como anomalia o dado de 2017-07-01.

In [ ]:
newdf_fonte1.iloc[24:36]

Analisando os dados de 2017, é possível observar que o dado de '2017-07-01' teve um variação bastante abrupta em relação aos meses anteriores e subsequentes, por isso o modelo o detectou como anômalo.


O próximo passo seria variar os parâmetros do modelo de autorregressão e verificar a robustez dessas descobertas, mas como isso pertence ao reino da análise de série temporal, vamos parar por aqui.

# Aplicar os modelos para uma outra fonte

Vamos agora valiar outra fonte: TOCHA HP/LP

In [ ]:
df_fonte2 = df_emissions.loc[df_emissions['14 - Fonte Emissora']=='TOCHA HP/LP']
df_fonte2['CO2e - AR4 (Mg)'].describe()

Esta fonte conta com 50 registros, média de 7484.996844, mínimo de 2635.808233 e máximo de 14950.602733.

In [ ]:
df_fonte2.head()

Agora, vamos separar apenas as colunas com os datos que precisamos analisar: 'Ano-Mes' e os dados de emissões 'CO2e - AR4 (Mg)'

In [ ]:
df_fonte2_ = pd.DataFrame(df_fonte2, columns=['Ano-Mes','CO2e - AR4 (Mg)'])

In [ ]:
df_fonte2_.head()

Para ler os dados no modelo proposto precisamos do arquivo no formato .csv e apenas com as colunas de data e emissões. 

In [ ]:
df_fonte2_.to_csv('fonte2.csv', index=False)

# Leitura e primeira análise gráfica dos dados 

In [ ]:
newdf_fonte2 = pd.read_csv('fonte2.csv', parse_dates=True, index_col=0)

In [ ]:
newdf_fonte2

In [ ]:
plt.figure(dpi=120)
newdf_fonte2.plot(ax=plt.gca())
plt.show();

Pelo gráfico plotado, é possível observar que os dados da fonte emissora ''TOCHA HP/LP' variam bastante entre os valores mínimos e máximos. Com os dados desta forma, não parece haver alguma anomalia bem como também parece ser muito difícil detectar caso haja. Vamos ver se os modelos identificam algo?

# Análise de Anomalias 2.1: Gráfico de Controle

Pelas estatísticas resumidas e no gráfico é difícil presumir se as anomalias estarão acima ou abaixo da média, que é de 7484.996844, com mínimo de 2635.808233 e máximo de 14950.602733. Portanto, precisamos  descobrir um gráfico que veja acima e abaixo)

Portanto, vamos criar um gráfico de controle unilateral (para baixo que era o único exemplo que tinha)

In [ ]:
def control_plot(time_series, threshold):
    '''
    Creates a one-sided control plot from a time series
    (that is, plots threshold above the mean but not below)
    Also, returns list of points that exceed the threshold
    i.e., points for which the value > mean + threshold*(standard deviation)
    
    Args: 
        time_series: (pandas dataframe; index column is date in datetime format and  
        column 0 is data)
        threshold: z-score threshold for anomaly detection (float)

    Returns: 
        Control plot of time_series    
        anomalies: anomalies that exceed threshold (pandas dataframe)
    ''' 
    
    mean_= time_series.iloc[:,0].mean()
    stdev_= time_series.iloc[:,0].std()
    cutoff = mean_+threshold*stdev_
    plt.figure(dpi=140)
    time_series.plot(ax=plt.gca())
    plt.axhline(y=mean_, color='g', linestyle='--', label='mean')
    # Use threshold to plot line at threshold*stdev_ times away from the mean
    plt.axhline(y=cutoff, color='r', linestyle=':', label='threshold')
    plt.legend(loc='best')
    plt.title('Fonte2')
    plt.ylabel('Emissões CO2eq')
    
    # Create dataframe of anomalies that exceed the cutoff
    anomalies = time_series[time_series.values > cutoff]
    return anomalies

Defina um limite de 3 desvios padrão e plote os resultados.

In [ ]:
anomaly_fonte2 = control_plot(newdf_fonte2, 3)

Pelo gráfico gerado, não foi detectada nenhuma anomalia na Fonte 2 = 'TOCHA HP/LP'. No entanto, como já indicado o precisávamos também avaliar para a parte de baixo da média dos dados.

In [ ]:
print(anomaly_fonte2)

# Análise de Anomalias 2.2: Soma acumulada (CUSUM)

In [ ]:
def cusum_plot(time_series, threshold):
    '''
    Plote as somas cumulativas alta e baixa e use-as para detecção de anomalias.
     Uma anomalia é relatada se as somas cumulativas estiverem além de um determinado limite.
    
     Args:
         time_series: (uma série temporal como pandas dataframe; a coluna de índice é date
         no formato datetime e a coluna 0 é data)
         threshold: limite para classificar o ponto como anomalia (float)

     Devoluções:
         Uma plotagem dos dados com a soma acumulada alta.
         cusum_results: as somas cumulativas alta e baixa, juntamente com dados
         e quaisquer anomalias acima e abaixo do limite (quadro de dados do pandas;
         da função cumsum)
    ''' 
    # Use a média e o desvio padrão de toda a série temporal
     # para calcular somas cumulativas
    mean_= time_series.iloc[:,0].mean()
    stdev_= time_series.iloc[:,0].std()
    
    # definir limiar em termos de desvio padrão
    cusum_results = cusum(time_series, mean_, stdev_, threshold*stdev_)
    ax=time_series.plot()
    ax.axhline(y=mean_, color='g', linestyle='--',label='average')
    ax.axhline(y=mean_+threshold*stdev_, color='r', linestyle='--',label='High threshold')
    ax.axhline(y=mean_-threshold*stdev_, color='r', linestyle='--',label='Low threshold')
    # Use threshold to plot line at threshold*stdev_ times away from the mean
    ax.scatter(x=cusum_results[0].index, y=cusum_results[0]['Low_Cusum'], 
             color='k', linestyle=':',label='Low Cusum')
    ax.scatter(x=cusum_results[0].index, y=cusum_results[0]['High_Cusum'], 
             color='y', linestyle=':',label='High Cusum')                 
    plt.legend(loc='lower right')
    plt.title('Fonte 2 Emissions - CUSUM')
    plt.ylabel('CO2 Emissions')
    plt.gcf().set_size_inches(8,6)
    plt.show()
    return cusum_results

Escolha do limiar: para o gráfico de controle, usamos 3 vezes o desvio padrão. Observe que o limite equivalente para CUSUM é * threshold * = 2, pois CUSUM inclui o * shift * (que escolhemos ser um desvio padrão) em seu cálculo da soma acumulada.

In [ ]:
anomaly_fonte2_cusum = cusum_plot(newdf_fonte2, 2);

Observe que HIGH CUSUM parece detectar maiores anomalias. O resultado reflete a diferença nos algoritmos de CUSUM e Tabela de Controle. A tabela de controle está procurando pontos únicos e anômalos. CUSUM é sensível a alterações no comportamento dos dados. Como resultado, sinaliza pontos como anômalos até que o comportamento da série temporal retorne ao normal.

De fato, CUSUM é usado para detecção de pontos de mudança: encontrar quando a distribuição subjacente da série temporal mudou.

In [ ]:
print(anomaly_fonte2_cusum)

Não consegui imprimir apenas os dados que apresentaram apenas anomalias. No gráfico da aula o professo também só teve o dado visual. Outra dificuldade não solucionada foi na plotagem do CUSUM em cima do gráfico de emissões.

# Análise 2.3: modelos autorregressivos

Nos dois métodos anteriores, realizamos a análise no modo offline - tínhamos todos os dados de interesse em mãos. Como resultado, ao examinar um determinado ponto, poderíamos usar o passado e o futuro (com relação a esse ponto) para calcular estatísticas como a média e o desvio padrão.

Modelos autorregressivos são comumente usados para detecção de anomalias em fluxo. Para analisar séries temporais no modo de streaming - ou seja, à medida que os dados se tornam disponíveis.

Como linha de base para o modelo de regressão automática, podemos verificar o que um gráfico de controle revela como anomalias. Em contraste com o exemplo da precipitação, aqui podemos ter anomalias em ambos os lados da média, portanto modificamos * control_plot * para se tornar um gráfico de controle completo de dois lados.

In [ ]:
def control_plot_full(time_series, threshold):
    '''
    Cria um gráfico de controle frente e verso de uma série temporal
     (limiar de plotagem acima e abaixo da média)
     Além disso, retorna a lista de pontos que excedem o limite
     ou seja, pontos para os quais o valor> média + limite * (desvio padrão)
     e valor <média - limiar * (desvio padrão)
    
     Args:
         time_series: (pandas dataframe; a coluna de índice é a data no formato datetime e
         coluna 0 são dados)
         threshold: limite para detecção de anomalias (float)

     Devoluções:
         Gráfico de controle de time_series
         anomalias: anomalias que excedem o limite (pandas dataframe)
    ''' 
    
    mean_= time_series.iloc[:, 0].mean()
    stdev_= time_series.iloc[:, 0].std()
    time_series.plot()
    plt.axhline(y=mean_, color='g', linestyle='--',label='average')
    # Use threshold to plot line at threshold*stdev_ times away from the mean
    plt.axhline(y=mean_+threshold*stdev_, color='r', linestyle=':', label='high threshold')
    plt.axhline(y=mean_-threshold*stdev_, color='m', linestyle=':', label='low threshold')
    plt.legend(loc='upper right')
    plt.title('Fonte 2 Emissions')
    plt.ylabel('CO2eq Emissions')
    plt.gcf().set_size_inches(8,6)
    plt.show()
    
    # Create dataframe of anomalies that exceed the threshold
    anomaly_mask = (np.abs(time_series.values - mean_) > threshold*stdev_)
    anomalies = time_series[anomaly_mask]
    return anomalies

In [ ]:
control_plot_full(newdf_fonte2, 3)

Somente pelo resultado do gráfico não encontramos nenhuma anomalia.

Observem que o low threshold não deveria ficar abaixo de 0. Todo dado abaixo de 0 deveria ser considerado uma anomalia, portanto o low threshold deve ser > 0.

In [ ]:
print(newdf_fonte2.index.inferred_freq)

O print acima serviu apenas para identificar a frequência dos registros: Frequência mensal

In [ ]:
newdf_fonte2.index.freq=newdf_fonte2.index.inferred_freq

Para a análise a seguir, tive que alterar a ordem de 1,1,4 para 1,1,2. Não entendi muito bem o que fiz, mas assim não deu mais erro.

In [ ]:
efonte2_sar=ARIMA(newdf_fonte2, order=(1,1,2)).fit()

efonte2_sar.summary()

Como no exemplo de aula, não discutiremos as estatísticas relatadas, exceto para dizer que ela inclui o desvio padrão dos resíduos (34.036	), que usaremos posteriormente.

Verificamos os diagnósticos para verificar se as premissas subjacentes ao modelo são atendidas e também para obter informações adicionais sobre a qualidade do ajuste. Isso é feito usando um gráfico Q-Q (verificando se os resíduos seguem uma distribuição normal), investigando os resíduos por padrões temporais e plotando um histograma dos resíduos

In [ ]:
# Get the predicted standard deviation. This is the 6.516 we saw earlier
sigma_pred = efonte2_sar.resid.std()
# Calculate the standardized residuals from the (regular) residuals
efonte2_std_resid = efonte2_sar.resid/sigma_pred

plt.title('Patterns in residual')
plt.plot(efonte2_std_resid);

In [ ]:
fig = plt.figure(figsize=(12,6))
ax = plt.subplot(121)
plt.title('Distribution of residuals')
sns.distplot(efonte2_std_resid.values, bins=50, ax=ax);
stats.probplot(efonte2_std_resid.values, dist='norm', sparams=(2.5,), plot=plt.subplot(122));

Para detecção de anomalias, focamos no gráfico superior: resíduos padronizados. O residual padronizado é o residual (a diferença entre o valor observado e o valor previsto) dividido pelo desvio padrão previsto (a raiz quadrada da variação prevista mencionada acima). É uma versão mais sofisticada do z-score.

Uma regra prática para detectar anomalias com resíduos padronizados: anomalias são pontos para os quais a magnitude dos resíduos padronizados é maior que 4. Vamos encontrar esses pontos.

In [ ]:
# Reportar as anomalias
anomaly_mask = np.abs(efonte2_std_resid) > 4
efonte2_anomalies = efonte2_std_resid[anomaly_mask]
print(efonte2_anomalies)

Nosso modelo não encontrou anomalias.

# Alterar os dados de uma fonte para ver se o modelo pega.

Agora, vou pegar o arquivo fonte2.csv e alterar manualmente dois dados de dois meses da seguinte forma:

2016-05-01 de 10882.292785568461 para 20882.292785568461

e

2018-05-01 de 4153.738730935139 para 0153.738730935139

que representariam "erros de digitação".

Aí vamos aplicar novamente os 3 modelos e ver se detecta estes dados anômalos. 

## Leitura e primeira análise gráfica dos dados 

In [ ]:
newdf_fonte2_teste = pd.read_csv('fonte2_teste.csv', parse_dates=True, index_col=0)

In [ ]:
newdf_fonte2_teste

In [ ]:
newdf_fonte2_teste['CO2e - AR4 (Mg)'].describe()

In [ ]:
plt.figure(dpi=120)
newdf_fonte2_teste.plot(ax=plt.gca())
plt.show();

Pelo gráfico plotado, é possível observar que os dados da fonte emissora ''TOCHA HP/LP' neste teste com dois registros alterados apresenta um pico em 2016 (máximo de 20882.292786 em 2016-05-01) e um vale em 2018 (mínimo de 153.738731 em 2018-05-01).  Vamos ver se os modelos identificam as anomalias?

## Análise de Anomalias 3.1: Gráfico de Controle

Portanto, vamos criar um gráfico de controle unilateral (para baixo que era o único exemplo que tinha)

In [ ]:
def control_plot(time_series, threshold):
    '''
    Creates a one-sided control plot from a time series
    (that is, plots threshold above the mean but not below)
    Also, returns list of points that exceed the threshold
    i.e., points for which the value > mean + threshold*(standard deviation)
    
    Args: 
        time_series: (pandas dataframe; index column is date in datetime format and  
        column 0 is data)
        threshold: z-score threshold for anomaly detection (float)

    Returns: 
        Control plot of time_series    
        anomalies: anomalies that exceed threshold (pandas dataframe)
    ''' 
    
    mean_= time_series.iloc[:,0].mean()
    stdev_= time_series.iloc[:,0].std()
    cutoff = mean_+threshold*stdev_
    plt.figure(dpi=140)
    time_series.plot(ax=plt.gca())
    plt.axhline(y=mean_, color='g', linestyle='--', label='mean')
    # Use threshold to plot line at threshold*stdev_ times away from the mean
    plt.axhline(y=cutoff, color='r', linestyle=':', label='threshold')
    plt.legend(loc='best')
    plt.title('Fonte2_teste')
    plt.ylabel('Emissões CO2eq')
    
    # Create dataframe of anomalies that exceed the cutoff
    anomalies = time_series[time_series.values > cutoff]
    return anomalies

Defina um limite de 3 desvios padrão e plote os resultados.

In [ ]:
anomaly_fonte2 = control_plot(newdf_fonte2_teste, 3)

Pelo gráfico gerado e resultado a seguir, foi detectada a anomalia superior de 20882.292786 do dia 2016-05-01. Como só temos o modelo de threshold superior,  não foi detectada a anomalias inferior.

In [ ]:
print(anomaly_fonte2)

### Análise de Anomalias 3.2: Soma acumulada (CUSUM)

In [ ]:
def cusum_plot(time_series, threshold):
    '''
    Plote as somas cumulativas alta e baixa e use-as para detecção de anomalias.
     Uma anomalia é relatada se as somas cumulativas estiverem além de um determinado limite.
    
     Args:
         time_series: (uma série temporal como pandas dataframe; a coluna de índice é date
         no formato datetime e a coluna 0 é data)
         threshold: limite para classificar o ponto como anomalia (float)

     Devoluções:
         Uma plotagem dos dados com a soma acumulada alta.
         cusum_results: as somas cumulativas alta e baixa, juntamente com dados
         e quaisquer anomalias acima e abaixo do limite (quadro de dados do pandas;
         da função cumsum)
    ''' 
    # Use a média e o desvio padrão de toda a série temporal
     # para calcular somas cumulativas
    mean_= time_series.iloc[:,0].mean()
    stdev_= time_series.iloc[:,0].std()
    
    # definir limiar em termos de desvio padrão
    cusum_results = cusum(time_series, mean_, stdev_, threshold*stdev_)
    ax=time_series.plot()
    ax.axhline(y=mean_, color='g', linestyle='--',label='average')
    ax.axhline(y=mean_+threshold*stdev_, color='r', linestyle='--',label='High threshold')
    ax.axhline(y=mean_-threshold*stdev_, color='r', linestyle='--',label='Low threshold')
    # Use threshold to plot line at threshold*stdev_ times away from the mean
    ax.scatter(x=cusum_results[0].index, y=cusum_results[0]['Low_Cusum'], 
             color='k', linestyle=':',label='Low Cusum')
    ax.scatter(x=cusum_results[0].index, y=cusum_results[0]['High_Cusum'], 
             color='y', linestyle=':',label='High Cusum')                 
    plt.legend(loc='lower right')
    plt.title('Fonte 2_teste Emissions - CUSUM')
    plt.ylabel('CO2 Emissions')
    plt.gcf().set_size_inches(8,6)
    plt.show()
    return cusum_results

Escolha do limiar: para o gráfico de controle, usamos 3 vezes o desvio padrão. Observe que o limite equivalente para CUSUM é * threshold * = 2, pois CUSUM inclui o * shift * (que escolhemos ser um desvio padrão) em seu cálculo da soma acumulada.

In [ ]:
anomaly_fonte2_teste_cusum = cusum_plot(newdf_fonte2_teste, 2);

Aparentemente nosso modelo detectou as duas anomalias inseridas bem como um outro registro (2017-07-01). O resultado reflete a diferença nos algoritmos de CUSUM e Tabela de Controle. A tabela de controle está procurando pontos únicos e anômalos. CUSUM é sensível a alterações no comportamento dos dados. Como resultado, sinaliza pontos como anômalos até que o comportamento da série temporal retorne ao normal.

De fato, CUSUM é usado para detecção de pontos de mudança: encontrar quando a distribuição subjacente da série temporal mudou.

In [ ]:
print(anomaly_fonte2_teste_cusum)

Não consegui imprimir apenas os dados que apresentaram apenas anomalias. No gráfico da aula o professo também só teve o dado visual. Outra dificuldade não solucionada foi na plotagem do CUSUM em cima do gráfico de emissões.

### Análise 3.3: modelos autorregressivos

Nos dois métodos anteriores, realizamos a análise no modo offline - tínhamos todos os dados de interesse em mãos. Como resultado, ao examinar um determinado ponto, poderíamos usar o passado e o futuro (com relação a esse ponto) para calcular estatísticas como a média e o desvio padrão.

Modelos autorregressivos são comumente usados para detecção de anomalias em fluxo. Para analisar séries temporais no modo de streaming - ou seja, à medida que os dados se tornam disponíveis.

Como linha de base para o modelo de regressão automática, podemos verificar o que um gráfico de controle revela como anomalias. Em contraste com o exemplo da precipitação, aqui podemos ter anomalias em ambos os lados da média, portanto modificamos * control_plot * para se tornar um gráfico de controle completo de dois lados.

In [ ]:
def control_plot_full(time_series, threshold):
    '''
    Cria um gráfico de controle frente e verso de uma série temporal
     (limiar de plotagem acima e abaixo da média)
     Além disso, retorna a lista de pontos que excedem o limite
     ou seja, pontos para os quais o valor> média + limite * (desvio padrão)
     e valor <média - limiar * (desvio padrão)
    
     Args:
         time_series: (pandas dataframe; a coluna de índice é a data no formato datetime e
         coluna 0 são dados)
         threshold: limite para detecção de anomalias (float)

     Devoluções:
         Gráfico de controle de time_series
         anomalias: anomalias que excedem o limite (pandas dataframe)
    ''' 
    
    mean_= time_series.iloc[:, 0].mean()
    stdev_= time_series.iloc[:, 0].std()
    time_series.plot()
    plt.axhline(y=mean_, color='g', linestyle='--',label='average')
    # Use threshold to plot line at threshold*stdev_ times away from the mean
    plt.axhline(y=mean_+threshold*stdev_, color='r', linestyle=':', label='high threshold')
    plt.axhline(y=mean_-threshold*stdev_, color='m', linestyle=':', label='low threshold')
    plt.legend(loc='upper right')
    plt.title('Fonte 2_teste Emissions')
    plt.ylabel('CO2eq Emissions')
    plt.gcf().set_size_inches(8,6)
    plt.show()
    
    # Create dataframe of anomalies that exceed the threshold
    anomaly_mask = (np.abs(time_series.values - mean_) > threshold*stdev_)
    anomalies = time_series[anomaly_mask]
    return anomalies

In [ ]:
control_plot_full(newdf_fonte2_teste, 3)

Observem que o low threshold não deveria ficar abaixo de 0. Todo dado abaixo de 0 deveria ser considerado uma anomalia, portanto o low threshold deve ser > 0.

Somente pelo resultado do gráfico, encontramos apenas a anomalias 2016-05-01	20882.292786.

In [ ]:
print(newdf_fonte2_teste.index.inferred_freq)

O print acima serviu apenas para identificar a frequência dos registros: Frequência mensal

In [ ]:
newdf_fonte2_teste.index.freq=newdf_fonte2_teste.index.inferred_freq

Para a análise a seguir, tive que alterar a ordem de 1,1,2 para 1,1,1. Não entendi muito bem o que fiz, mas assim não deu mais erro.

In [ ]:
efonte2_teste_sar=ARIMA(newdf_fonte2_teste, order=(1,1,1)).fit()

efonte2_teste_sar.summary()

Como no exemplo de aula, não discutiremos as estatísticas relatadas, exceto para dizer que ela inclui o desvio padrão dos resíduos (32.300), que usaremos posteriormente.

Verificamos os diagnósticos para verificar se as premissas subjacentes ao modelo são atendidas e também para obter informações adicionais sobre a qualidade do ajuste. Isso é feito usando um gráfico Q-Q (verificando se os resíduos seguem uma distribuição normal), investigando os resíduos por padrões temporais e plotando um histograma dos resíduos

In [ ]:
# Get the predicted standard deviation. This is the 6.516 we saw earlier
sigma_pred = efonte2_teste_sar.resid.std()
# Calculate the standardized residuals from the (regular) residuals
efonte2_teste_std_resid = efonte2_teste_sar.resid/sigma_pred

plt.title('Patterns in residual')
plt.plot(efonte2_teste_std_resid);

In [ ]:
fig = plt.figure(figsize=(12,6))
ax = plt.subplot(121)
plt.title('Distribution of residuals')
sns.distplot(efonte2_teste_std_resid.values, bins=50, ax=ax);
stats.probplot(efonte2_teste_std_resid.values, dist='norm', sparams=(2.5,), plot=plt.subplot(122));

Para detecção de anomalias, focamos no gráfico superior: resíduos padronizados. O residual padronizado é o residual (a diferença entre o valor observado e o valor previsto) dividido pelo desvio padrão previsto (a raiz quadrada da variação prevista mencionada acima). É uma versão mais sofisticada do z-score.

Uma regra prática para detectar anomalias com resíduos padronizados: anomalias são pontos para os quais a magnitude dos resíduos padronizados é maior que 4. Vamos encontrar esses pontos.

In [ ]:
# Reportar as anomalias
anomaly_mask = np.abs(efonte2_teste_std_resid) > 4
efonte2_teste_anomalies = efonte2_teste_std_resid[anomaly_mask]
print(efonte2_teste_anomalies)

Nosso modelo encontrou apenas a anomalia de 2016-05-01.

# Questões a resolver

1) Notebook está sendo criado no colab e tenho que ficar colocando a base de dados toda vez q desconecto e conecto novamente. Qual seriam as soluçõs alternativas? 
  a) colocar no meu drive mas aí teria que dar acesso ao novo usuário?
  b) colocar no git hub e ler de lá?

2) Análise de Anomalias 1.1: Gráfico de Controle - Não consegui colocar o limite inferior, apenas o superior que foi o mesmo utilizado no exemplo de aula pelo professor.

3) Análise de Anomalias 1.2: CUSUM - Não consegui analisar/plotar o scatter fora da linha 0:

    ax.scatter(x=cusum_results[0].index, y=cusum_results[0]['Low_Cusum'], 
             color='k', linestyle=':',label='Low Cusum')
    ax.scatter(x=cusum_results[0].index, y=cusum_results[0]['High_Cusum'], 
             color='y', linestyle=':',label='High Cusum') 

Não consegui printar apenas os dados com anomalias detectadas.

4) Análise 3.3: modelos autorregressivos

Observem que o low threshold não deveria ficar abaixo de 0. Todo dado abaixo de 0 deveria ser considerado uma anomalia, portanto o low threshold deve ser > 0.

Apenas alterando a magnitude dos resíduos para >2 é detectada a anomalia de 2018-05-01, no entanto ele detecta também outro ponto que não seria anômalo.